In [1]:
%matplotlib inline
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
n = 3
num_vecs = 50000
block = np.array([[0,-1],[1,0]])
J = np.kron(np.eye(n, dtype=int),block)
np.random.seed(99)
true_U = np.random.rand(2*n,2*n)
true_U, _ = np.linalg.qr(true_U) #U should be orthonormal
vecs = np.random.rand(num_vecs, 2*n) #generate 500 embedding vectors
cur_index = np.random.randint(0,num_vecs)
true_matrix = true_U @ J @ true_U.T

In [7]:
(np.array([5,1]))@ block @ (np.array([2,1]))

-6

In [ ]:
weights = vecs @ (vecs @ true_matrix).T
probs = 1 / (1+np.exp(-weights))
maxes = np.max(probs, axis=1)
best_one = np.argmin(maxes)
best_one

In [13]:
sorted(probs[best_one], reverse=True)[:5]

[0.5,
 0.4851165718081027,
 0.4748070045917384,
 0.47096977174285604,
 0.4689651008177996]

In [14]:
altmaxes = np.average(probs, axis=1)
other_one = np.argmin(altmaxes)
print(other_one)
sorted(probs[other_one], reverse=True)[:5]

344


[0.5781041097461399,
 0.5262474760213333,
 0.5248832537340133,
 0.5243914493100928,
 0.5062967442521352]

In [9]:
#check ones that are definitively positive and not noise, should add noise check later
max_prob = 0.6
adjacency = (probs > max_prob).astype(int)
G = nx.from_numpy_array(adjacency, create_using=nx.DiGraph)
SCC = list(nx.strongly_connected_components(G))
component = list(max(SCC, key=len))

In [10]:
list(map(len, SCC))

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
cur_node = np.random.randint(0, num_vecs)
prev_max = 1
for i in range(50):
    weights = vecs @ true_matrix @ vecs[cur_node]
    probs = 1 / (1+np.exp(-weights))
    cur_max = np.max(probs)
    if cur_max < prev_max:
        prev_max = cur_max
        cur_node = next_node
    else: break

In [ ]:
def get_graph_data(inner_products, threshold, layout='spring', fixed_positions=None):
    """
    Given a full inner_products matrix, return:
      - positions: a dict mapping node index to (x, y) coordinates.
      - edges: a list of tuples ((x1, y1), (x2, y2)) representing directed edges,
        where an edge from i to j exists if inner_products[i, j] > threshold.
    
    Parameters:
        inner_products : numpy.ndarray
            A (n, n) matrix where element (i, j) is the weight ⟨R v_j, v_i⟩.
        threshold : float
            Only edges with weight greater than threshold are retained.
        layout : str, optional
            The layout algorithm to use for positioning (default 'spring').
            Other options can be passed to networkx's layout functions.
        fixed_positions : dict, optional
            If provided, a dictionary mapping node indices to positions (x, y).
            If given, these positions are used instead of computing a new layout.
    
    Returns:
        positions : dict
            Mapping from node index to (x, y) coordinates.
        edges : list of tuple
            Each tuple is ((x1, y1), (x2, y2)) for an edge from node i to j.
    """
    n = inner_products.shape[0]
    # Build a directed graph with all nodes
    G = nx.DiGraph()
    G.add_nodes_from(range(n))

    # Add an edge from i -> j if the weight exceeds the threshold.
    # You can modify this condition to include additional edge properties if needed.
    for i in range(n):
        for j in range(n):
            if inner_products[i, j] > threshold:
                G.add_edge(i, j, weight=inner_products[i, j])

    # Compute positions if not provided
    if fixed_positions is not None:
        positions = fixed_positions
    else:
        if layout == 'spring':
            positions = nx.spring_layout(G)
        elif layout == 'circular':
            positions = nx.circular_layout(G)
        elif layout == 'kamada_kawai':
            positions = nx.kamada_kawai_layout(G)
        else:
            # Default to spring layout if unknown
            positions = nx.spring_layout(G)

    # Extract edges as line segments using the computed positions.
    edges = []
    for i, j in G.edges():
        # Each edge is represented as a segment from positions[i] to positions[j]
        edges.append((positions[i], positions[j]))

    return positions, edges

# Example usage:
# Suppose you have an inner_products matrix computed earlier
# inner_products = V @ (V @ R.T).T  # Your computed inner products
# threshold = 0.1  # This is your decay parameter value
# pos, edge_segments = get_graph_data(inner_products, threshold)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.animation import FuncAnimation

# Assume inner_products is computed and fixed_positions is computed once:
fixed_positions, _ = get_graph_data(probs, threshold=-np.inf, layout='circular')  # get layout for all nodes

fig, ax = plt.subplots()
# Plot nodes using fixed positions
nodes_coords = np.array([fixed_positions[i] for i in range(len(fixed_positions))])
scatter = ax.scatter(nodes_coords[:, 0], nodes_coords[:, 1], color='blue')
edge_collection = LineCollection([], color='black', linewidths=1)
ax.add_collection(edge_collection)

def update(frame):
    # Update threshold for this frame (decaying threshold)
    k = 0.9 - (frame/num_frames)/10 # goes from 0.9-0.5

    # Get filtered edges using the precomputed fixed_positions
    _, current_edges = get_graph_data(probs, k, fixed_positions=fixed_positions)

    # Update the edge collection for drawing
    edge_collection.set_segments(current_edges)
    ax.set_title(f"Decay threshold: {k:.3f}")
    return scatter, edge_collection

# Configure animation parameters as needed:
initial_threshold = 0.5   # example starting threshold
decay_factor = 0.95       # threshold decay factor per frame
num_frames = 120

ani = FuncAnimation(fig, update, frames=range(num_frames), blit=True)

In [ ]:
ani.save("graph.gif", writer='pillow', fps=10)